In [1]:
#Import modules
import pandas as pd
from sqlalchemy import create_engine 

In [2]:
#EXTRACT

#Get breweries data
brew_file = "data/breweries_us.csv"
brew_df = pd.read_csv(brew_file)
brew_df.head()

,brewery_name,type,address,website,state,state_breweries
0,Valley Brewing Co.,Brewpub,"PO Box 4653, Stockton, California, 95204",http://www.valleybrew.com/,california,284
1,Valley Brewing Co.,Brewpub,"157 Adams St., Stockton, California, 95204",http://www.valleybrew.com/,california,284
2,Valley Brewing Co,Microbrewery,"1950 W Freemont, Stockton, California, 95203",http://www.valleybrew.com/,california,284
3,Ukiah Brewing Company,Brewpub,"102 S. State St., Ukiah, California, 95482",http://www.ukiahbrewingco.com/,california,284
4,Tustin Brewing Co.,Brewpub,"13011 Newport Ave. #100, Tustin, California, 9...",http://www.tustinbrewery.com/,california,284


In [3]:
#Get census data
census_file = "data/census_pop.csv"
census_df = pd.read_csv(census_file)
census_df.head()

,SUMLEV,REGION,DIVISION,STATE,NAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,POPESTIMATE2011,POPESTIMATE2012,...,RDOMESTICMIG2017,RDOMESTICMIG2018,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015,RNETMIG2016,RNETMIG2017,RNETMIG2018
0,10,0,0,0,United States,308745538,308758105,309326085,311580009,313874218,...,0.000000,0.000000,2.553948,2.746049,2.701727,2.988276,3.328598,3.321549,2.941086,3.001086
1,20,1,0,0,Northeast Region,55317240,55318430,55380645,55600532,55776729,...,-5.651919,-5.222289,0.845134,0.040762,-0.397011,-0.923951,-2.011735,-2.430459,-1.801582,-1.127222
2,20,2,0,0,Midwest Region,66927001,66929743,66974749,67152631,67336937,...,-2.370672,-2.301663,-1.043009,-0.896575,0.042505,-0.715656,-1.357662,-1.226811,-0.519621,-0.431833
3,20,3,0,0,South Region,114555744,114563045,114867066,116039399,117271075,...,2.963135,2.779373,5.379667,5.836112,5.290067,6.206402,7.328494,7.225046,6.252425,6.148925
4,20,4,0,0,West Region,71945553,71946887,72103625,72787447,73489477,...,1.478565,1.350094,2.689358,3.226360,3.343874,4.148127,5.127995,5.372314,4.164981,3.965769


In [4]:
#TRANSFORM

#Select the needed columns from brew_df
brew_df = brew_df[['brewery_name', 'type', 'address', 'state']]
brew_df.head()

,brewery_name,type,address,state
0,Valley Brewing Co.,Brewpub,"PO Box 4653, Stockton, California, 95204",california
1,Valley Brewing Co.,Brewpub,"157 Adams St., Stockton, California, 95204",california
2,Valley Brewing Co,Microbrewery,"1950 W Freemont, Stockton, California, 95203",california
3,Ukiah Brewing Company,Brewpub,"102 S. State St., Ukiah, California, 95482",california
4,Tustin Brewing Co.,Brewpub,"13011 Newport Ave. #100, Tustin, California, 9...",california


In [5]:
#Replace the dashes for spaces to match the census data
brew_df['state'] = brew_df['state'].str.replace('-', ' ')

#Name the index 'id'
brew_df.index.name = 'id'
brew_df.head()

,brewery_name,type,address,state
id,,,,
0,Valley Brewing Co.,Brewpub,"PO Box 4653, Stockton, California, 95204",california
1,Valley Brewing Co.,Brewpub,"157 Adams St., Stockton, California, 95204",california
2,Valley Brewing Co,Microbrewery,"1950 W Freemont, Stockton, California, 95203",california
3,Ukiah Brewing Company,Brewpub,"102 S. State St., Ukiah, California, 95482",california
4,Tustin Brewing Co.,Brewpub,"13011 Newport Ave. #100, Tustin, California, 9...",california


In [6]:
#Select the needed columns from the census data
census_df = census_df[["STATE", "NAME", "POPESTIMATE2018"]]

#Rename the column headers
census_df = census_df.rename(columns={"STATE": "is_state",
                                    "NAME": "state",
                                    "POPESTIMATE2018": "population_estimate"})

#Remove the rows that aren't states
census_df = census_df[census_df.is_state != 0]

In [7]:
#Dropped the column that identified the states
census_df = census_df.drop(columns="is_state")

#Make all the state's names lowercase to match the brewery data
census_df['state'] = census_df['state'].str.lower()

In [8]:
#Renamed Washington DC to match the brewery data
census_df["state"] = census_df["state"].str.replace("district of columbia", "washington dc")

#Reset index values
census_df = census_df.reset_index().drop(columns="index")

#Name the index 'id'
census_df.index.name = 'id'
census_df.head()

,state,population_estimate
id,,
0,alabama,4887871
1,alaska,737438
2,arizona,7171646
3,arkansas,3013825
4,california,39557045


In [9]:
#LOAD

#Connect to local database
rds_connection_string = "username:password@localhost:5432/breweries_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [11]:
#Check for tables
engine.table_names() 

['brewery', 'state_pop']

In [12]:
#Send data to postgresql
brew_df.to_sql(name='brewery', con=engine, if_exists='append', index=True)
census_df.to_sql(name='state_pop', con=engine, if_exists='append', index=True)

In [13]:
#Check if data was added to postgres
pd.read_sql_query('select * from brewery', con=engine).head()

,id,brewery_name,type,address,state
0,0,Valley Brewing Co.,Brewpub,"PO Box 4653, Stockton, California, 95204",california
1,1,Valley Brewing Co.,Brewpub,"157 Adams St., Stockton, California, 95204",california
2,2,Valley Brewing Co,Microbrewery,"1950 W Freemont, Stockton, California, 95203",california
3,3,Ukiah Brewing Company,Brewpub,"102 S. State St., Ukiah, California, 95482",california
4,4,Tustin Brewing Co.,Brewpub,"13011 Newport Ave. #100, Tustin, California, 9...",california


In [14]:
#Check if data was added to postgres
pd.read_sql_query('select * from state_pop', con=engine).head()

,id,state,population_estimate
0,0,alabama,4887871
1,1,alaska,737438
2,2,arizona,7171646
3,3,arkansas,3013825
4,4,california,39557045
